# Анализ музыкальных предпочтений жителей Москвы и Санкт-Петербурга

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки мы сравним поведение пользователей двух столиц.

**Цель исследования** — проверим три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей мы получим из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

Составим первое представление о данных Яндекс.Музыки.

Основной инструмент аналитика — `pandas`. Импортируем эту библиотеку.

In [1]:
# импорт библиотеки pandas
import pandas as pd

Прочитаем файл `yandex_music_project.csv` из папки `/datasets` и сохраним его в переменной `df`:

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv("/datasets/yandex_music_project.csv")

Выведем на экран первые десять строк таблицы:

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Одной командой получим общую информацию о таблице:

In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны нарушения хорошего стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Для названий столбцов рекомендуется использовать "змеиный_регистр".

Количество значений в столбцах различается. Значит, в данных есть пропуски.

**Выводы**

- В каждой строке таблицы — данные о прослушанном треке. 
- Часть колонок описывает саму композицию: название, исполнителя и жанр. 
- Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски, а в названиях колонок расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить выявленные проблемы.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков
Выведем на экран названия столбцов:

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведём названия в соответствие с хорошим стилем:
* несколько слов в названии запишим в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы,
* конкретизируем названия столбцов.

Для этого переименуем колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'`    → `'track_name'`;
* `'artist'`   → `'artist_name'`;
* `'genre'`    → `'genre_name'`;
* `'  City  '` → `'user_city'`;
* `'time'`     → `'start_time'`;
* `'Day'`      → `'day_of_week'`;
.

In [6]:
# переименование столбцов
df = df.rename(columns={'  userID': 'user_id',
                        'Track': 'track_name',
                        'artist' : 'artist_name',
                        'genre' : 'genre_name',
                        '  City  ': 'user_city',
                        'time' : 'start_time',
                        'Day' : 'day_of_week' })

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [7]:
# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'user_city',
       'start_time', 'day_of_week'],
      dtype='object')

### Пропуски значений
Сначала посчитаем сколько в таблице пропущенных значений. 

Для этого достаточно двух методов `pandas`:

In [8]:
# подсчёт пропусков
df.isna().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
user_city         0
start_time        0
day_of_week       0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track_name` и `artist_name` пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre_name` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. 

Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track_name`, `artist_name` и `genre_name` на строку `'unknown'`. 

Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track_name', 'artist_name', 'genre_name']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [10]:
# подсчёт пропусков
df.isna().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
user_city      0
start_time     0
day_of_week    0
dtype: int64

### Дубликаты
Посчитаем явные дубликаты в таблице одной командой:

In [11]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

Вызовим специальный метод `pandas`, чтобы удалить явные дубликаты:

In [12]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True) 

Ещё раз посчитаем явные дубликаты в таблице — убедимся, что полностью от них избавились:

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Проверим неявные дубликаты в столбце `user_city`:

In [14]:
df['user_city'].sort_values().unique()

array(['Moscow', 'Saint-Petersburg'], dtype=object)

Неявные дубликаты в столбце `user_city` отсутствуют. 

Теперь проверим неявные дубликаты в столбце `day_of_week`:

In [15]:
df['day_of_week'].sort_values().unique()

array(['Friday', 'Monday', 'Wednesday'], dtype=object)

Неявные дубликаты в столбце `day_of_week` отсутствуют. Данные представлены для 3 дней: понедельник, среда, пятница.


Теперь избавимся от неявных дубликатов в колонке `genre_name`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлечем нужный столбец датафрейма, 
* применим к нему метод сортировки,
* для отсортированного столбца вызовем метод, который вернёт уникальные значения из столбца.

In [16]:
# Просмотр уникальных названий жанров
df['genre_name'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Выведем также общее количество уникальных названий:

In [17]:
len(df['genre_name'].sort_values().unique())

290

Просмотрим список и найдем неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Мы видим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

Также неявным дубликатом является название `электроника` на русском языке. Заменим его на `electronic`.

Подозрения вызывают жанры `türk`, `türkçe`, указанные не на английском языке и жанр `ïîï`. Так как нет возможности уточнить смысл этих жанров, оставим их в исходном состоянии.

Процент треков жанра `türk` от общего количества треков:

In [18]:
print('{:.4%}'.format((df[df['genre_name']=='türk']['user_id'].count())/len(df['genre_name'])))

0.0065%


Процент треков жанра `türkçe` от общего количества треков:

In [19]:
print('{:.4%}'.format((df[df['genre_name']=='türkçe']['user_id'].count())/len(df['genre_name'])))

0.0408%


Процент треков жанра `ïîï` от общего количества треков:

In [20]:
print('{:.4%}'.format((df[df['genre_name']=='ïîï']['user_id'].count())/len(df['genre_name'])))

0.0016%


Очевидно, что они не несут рисков для качества исследования.

Вернемся к функции для исправления жанров. Сделаем так, чтобы списку из `wrong_genres` соответствовало одно значение из `correct_genre`. Тогда не нужно создавать дополнительные переменные и несколько раз вызывать функцию `replace_wrong_genres`:

In [21]:
#Функция для замены неявных дубликатов
def replace_wrong_genres(column_name, wrong_genres, correct_genre):
    #итерирруемся по wrong_genres
    for i in range(len(wrong_genres)):
        #шагаем по названиям, которые требуют исправления, и исправляем
        for j in range(len(wrong_genres[i])):
            column_name = column_name.replace(wrong_genres[i][j], correct_genre[i])
    return column_name

Вызовем `replace_wrong_genres()` и передим ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [22]:
# Устранение неявных дубликатов
wrong_genres = [['hip','hop','hip-hop'],
                ['электроника']]
correct_genre = ['hiphop',
                 'electronic']

df['genre_name'] = replace_wrong_genres(df['genre_name'], wrong_genres, correct_genre)

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведем отсортированный список уникальных значений столбца `genre`:

In [23]:
# Проверка на неявные дубликаты
df['genre_name'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Проверим общее количество уникальных названий в столбце `genre_name`, оно должно стать меньше на 4 позиции:

In [24]:
len(df['genre_name'].sort_values().unique())

286

И, действительно, работает как ожидаем.

### Достаточность данных по периодам "утро" и "вечер"

Данные за какой период времени суток в нашем распоряжении:

In [25]:
#самое позднее значение времени
df['start_time'].max()

'22:00:59'

In [26]:
#самое раннее значение времени
df['start_time'].min()

'08:00:00'

Создадим функцию для проверки количества данных в заданном интервале времени:

In [27]:
#функция для проверки количества данных в заданном 
#интервале времени
def count_interval(time1,time2):
    tracks_in_interval = (len(df['start_time']) - 
                          df[df['start_time']<time1]['user_id'].count()-
                          df[df['start_time']>time2]['user_id'].count())
    return tracks_in_interval

Создадим массив для последующего расчета по интервалам времени:

In [28]:
#создаём массив для последующего расчета по интервалам времени
time_for_intervals = ['07:00','08:00','09:00','10:00','11:00','12:00','13:00', '14:00', '15:00', '16:00',
                     '17:00', '18:00' ,'19:00' ,'20:00' ,'21:00','22:01','23:00']

Введём счетчик количества треков, чтобы не потерять ни одного при разделении треков по промежуткам времени:

In [29]:
#счетчик для проверки правильности разделения по периодам
total_tracks = 0 

Теперь воспользуемся этой функцией внутри цикла:

In [30]:
#рассчитаем количество треков по периодам и выведем на экран
for i in range(len(time_for_intervals)-1):
    total_tracks  += count_interval(time_for_intervals[i],time_for_intervals[i+1])
    print('Количество треков с', time_for_intervals[i], 'до', time_for_intervals[i+1], ':',
          count_interval(time_for_intervals[i],time_for_intervals[i+1]))

Количество треков с 07:00 до 08:00 : 0
Количество треков с 08:00 до 09:00 : 9512
Количество треков с 09:00 до 10:00 : 9511
Количество треков с 10:00 до 11:00 : 169
Количество треков с 11:00 до 12:00 : 0
Количество треков с 12:00 до 13:00 : 0
Количество треков с 13:00 до 14:00 : 10304
Количество треков с 14:00 до 15:00 : 10722
Количество треков с 15:00 до 16:00 : 176
Количество треков с 16:00 до 17:00 : 0
Количество треков с 17:00 до 18:00 : 0
Количество треков с 18:00 до 19:00 : 0
Количество треков с 19:00 до 20:00 : 0
Количество треков с 20:00 до 21:00 : 10829
Количество треков с 21:00 до 22:01 : 10030
Количество треков с 22:01 до 23:00 : 0


Проверим корректность подсчета количества треков по периодам времени:

In [31]:
print('Общее количество треков по периодам времени:', total_tracks )
print('Общее количество треков в таблице после предобработки:', len(df['user_id']))

Общее количество треков по периодам времени: 61253
Общее количество треков в таблице после предобработки: 61253


Очевидно, что разбивка по периодам выполнена верно. 

Если нам предстоит исследовать периоды "утро" и "вечер", то следует учесть, что по вечернему периоду есть данные только с 21:00 до 22:01. Данные за утренний период даны с 08:00 до 11:00, что лучше, чем для вечернего периода. 

Вряд ли в периоды, для которых данных нет, люди не слушают музыку. Вероятно массив исходных данных неполный. Для более расширенного анализа утреннего и вечернего периодов желательно дополнить исходные данные:
- утром - за период 07:00-08:00,
- вечером - за период 17:00-21:00,

так как это "часы пик", в которые количество прослушиваний музыки может возрасти и изменить результаты исследования. Также разделение на периоды "утро" и "вечер" будет одинаковым по протяжённости и составит 4 часа (07-11, 17-21).

Похоже, что датасет готовили под проект, и в первоначальном варианте задание могло быть немного другим.

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Также проанализировано количество доступных для анализа треков по часам дня. В вечерний период есть треки с 20 до 22 ч. В утренние часы доступны треки с 8 до 11 ч. Данные можно было бы улучшить дополнительной информацией по другим часам, но на данный момент такая возможность отсутствует.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre_name`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполним каждый из расчётов по отдельности. 

Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [32]:
# Подсчёт прослушиваний в каждом городе
display(df.groupby('user_city')['genre_name'].count()) #сколько треков прослушано за пн, ср, пт

user_city
Moscow              42741
Saint-Petersburg    18512
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. Учтём, что в данных есть информация о прослушиваниях только за эти дни.


In [33]:
# Подсчёт прослушиваний по дням недели
display(df.groupby('day_of_week')['genre_name'].count())

day_of_week
Friday       21840
Monday       21354
Wednesday    18059
Name: genre_name, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Мы увидели, как работает группировка по городу и по дням недели. Теперь напишем функцию, которая объединит два эти расчёта.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраним в переменную строки исходной таблицы, у которых значение:
  * в колонке `day_of_week` равно параметру `day`,
  * в колонке `user_city` равно параметру `city`.

Для этого применим последовательную фильтрацию с логической индексацией.

Затем посчитаем значения в столбце `user_id` получившейся таблицы. Результат сохраним в новую переменную. Вернём эту переменную из функции.

In [34]:
# Объявляется функция с двумя параметрами: day, city.
def number_tracks(day, city):
    
    # В переменной track_list сохраняются те строки таблицы df, для которых 
    # значение в столбце 'day_of_week' равно параметру day и одновременно значение
    # в столбце 'user_city' равно параметру city
    track_list = df[df['day_of_week'] == day]
    track_list = track_list[track_list['user_city'] == city]
    
    # В переменной track_list_count сохраняется число значений столбца 'user_id',
    # рассчитанное методом count() для таблицы track_list.
    track_list_count = track_list['user_id'].count()
    
    # Функция возвращает число - значение track_list_count. 
    return track_list_count

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [35]:
print('Количество прослушиваний:')
print()
for city in ['Moscow','Saint-Petersburg']:
    for day in ['Monday','Wednesday','Friday']:
        print(city,',', day, ':', number_tracks(day, city))
        print()

Количество прослушиваний:

Moscow , Monday : 15740

Moscow , Wednesday : 11056

Moscow , Friday : 15945

Saint-Petersburg , Monday : 5614

Saint-Petersburg , Wednesday : 7003

Saint-Petersburg , Friday : 5895



Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [36]:
# Таблица с результатами
new_colums_1 = ['city', 'monday', 'wednesday', 'friday']

data_1 = [['Moscow', 15740,11056,15945],
          ['Saint-Petersburg',5614,7003,5895]]

pd.DataFrame(data=data_1, columns=new_colums_1)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Другой способ**

Посчитаем количество треков по дням недели в городах с использованием встроенных функций.

Разделим пользователей Санкт-Петербурга и Москвы с использованием логической индексации:

In [37]:
#фильтрация пользователей по городам логической индексацией

# получение таблицы moscow_general из тех строк таблицы df,
# для которых значение в столбце 'user_city' равно 'Moscow'
moscow_general = df[df['user_city']=='Moscow']

# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'user_city' равно 'Saint-Petersburg'
spb_general = df[df['user_city']=='Saint-Petersburg']

Количество треков *по дням недели* **в Москве**:

In [38]:
print('Moscow')
moscow_general.groupby('day_of_week')['user_id'].count().sort_values(ascending=False)

Moscow


day_of_week
Friday       15945
Monday       15740
Wednesday    11056
Name: user_id, dtype: int64

Количество треков *по дням недели* **в Санкт-Петербурге**:

In [39]:
print('Saint-Petersburg')
spb_general.groupby('day_of_week')['user_id'].count().sort_values(ascending=False)

Saint-Petersburg


day_of_week
Wednesday    7003
Friday       5895
Monday       5614
Name: user_id, dtype: int64

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

В предыдущем разделе было создано две переменные с данными по городам:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [40]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
def genre_weekday(table, day, time1, time2):
    
    # 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
    #    которых одновременно:
    genre_df = table[table['day_of_week']==day]         #    - значение в столбце day равно значению аргумента day
    genre_df = genre_df[genre_df['start_time']>time1]   #    - значение в столбце time больше значения аргумента time1
    genre_df = genre_df[genre_df['start_time']<time2]   #    - значение в столбце time меньше значения аргумента time2
    
    # 2) группируем датафрейм genre_df по столбцу genre, взяв один из его столбцов и
    #    посчитаем методом count() количество записей для каждого из жанров
    genre_df_count = genre_df.groupby('genre_name')['user_id'].count()
    
    # 3) отсортируем genre_df_count по убыванию встречаемости и сохраним
    #    в переменную genre_df_sorted
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    
    # 4) вернем Series из 10 первых значений genre_df_sorted, это будут топ-10
    #    популярных жанров (в указанный день, в заданное время)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [41]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [42]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [43]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [44]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre_name
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

**Следует учесть**, что в исходных данных представлены треки  за периоды времени 08:00-11:00 (утро) и 21:00-22:00 (вечер). Для более полного сравнения желательно получить данные за периоды 07:00-08:00 и 17:00-21:00. Тогда сравнение можно было сделать по "часам пик": 07:00-11:00 и 17:00-21:00. Это были бы равные промежутки времени по 4 часа утром и вечером. Для вечера пятницы можно было расширить вечерний период: 17:00-23:00.

Ещё более корректно взять полные сутки, чтобы наверняка определить часы пик.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  

А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [45]:
# группировка таблицы moscow_general по 'genre_name', подсчёт числа значений 'genre_name', 
# сортировка получившегося Series в порядке убывания
moscow_genres = moscow_general.groupby('genre_name')['user_id'].count().sort_values(ascending = False)

Выведем на экран первые десять строк `moscow_genres`:

In [46]:
# просмотр первых 10 строк moscow_genres
display(moscow_genres.head(10))

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: user_id, dtype: int64

Теперь повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:


In [47]:
# группировка таблицы spb_general по столбцу 'genre_names', подсчёт числа значений 'genre_names'
# сортировка получившегося Series в порядке убывания
spb_genres = spb_general.groupby('genre_name')['user_id'].count().sort_values(ascending = False)

Выведем на экран первые десять строк `spb_genres`:

In [48]:
# просмотр первых 10 строк spb_genres
display(spb_genres.head(10))

genre_name
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: user_id, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**Следует учесть**, что в исходных данных для периодов "утро" и "вечер" представлены треки за промежутки времени 08:00-11:00 (утро) и 21:00-22:00 (вечер). Для более расширенного анализа желательно получить дополнительные данные, о чем подробнее сказано в выводах к проверке 2-ой гипотезы.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных.